# 📓 Custom App using DummyAPI

The files in this folder code a custom app that resembles an LLM app with
seperate classes for common app components. None of these components make
network calls but pretend to otherwise operate like real components.

__WARNING__: This example must be run from the git repository as it imports
files that are not included in a PIP distribution.

In [ ]:
from pathlib import Path
import sys

# If using trulens from the repository, add the parent directory to the path:
sys.path.append(str(Path().cwd().parent.parent.parent.parent.resolve()))

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_agent import CustomAgent

from trulens_eval import Tru
from trulens_eval.tru_custom_app import TruCustomApp

tru = Tru()
tru.reset_database()

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
# Show instrumented components and methods.

ta.print_instrumented()

In [ ]:
# Normal usage (without tracing):
ca.respond_to_query(query="What is the capital of Indonesia?")

# Instrumented usage:
with ta as recorder:
    response = ca.respond_to_query(query="What is the capital of Indonesia?")

record = recorder.get()

In [ ]:
# Show the app output:

# response

In [ ]:
# Show the instrumentation record.

# record.model_dump()

In [ ]:
# Start the dasshboard. If you running from github repo, include the _dev arg below:
tru.start_dashboard(
    _dev=Path().cwd().parent.parent.parent.parent.resolve(),
    force=True
)

# Custom App options

The custom app used above has some configuration options or call variants with
differing behaviours:

- Number of agents and number of tools is configurable via `CustomApp` class
  params/attributes `num_agents` and `num_tools`.

- Simulated delay and memory usage controlled by `Dummy` class `delay` and
  `alloc` attributes. All other components subtype `Dummy` hence also implement
  simulated delays and allocations.

- Non-determinism control via the `Dummy` class `seed` attribute. Randomness in
  some components can be somewhat controlled by this attribute. The control is
  presently incomplete as threading and async usage (described below) is not
  controlled by `seed`.

- Async methods. These begin with the letter "a" followed by the sync method
  name.

- Streaming methods. The main method to app and the generation methods to the
  llm component each have streaming versions (`stream_respond_to_query` and
  `stream`). These produce iterables over strings instead of single strings. The
  async variants add a prefix "a" to these and produce asynchronous iterables.

- Use of parallelism for processing retrieved chunks. This uses either threads
  or async depending on which method variant is called. This is controlled by
  the `CustomApp` class `use_parallel` flag.

- Nested use of custom app inside a custom app component. This is controlled by
  the `CustomAgent` class `use_app` flag.

- Use of trulens_eval to record the invocation of the above nested model. This
  is controlled by the `CustomAgent` class `use_recorder` flag.



In [ ]:
# This cell executes the app in the various configuration options describe
# above. The final app output should be identical in all cases.

for use_parallel in [False, True]:
    for use_app in [False, True]:
        for use_recorder in [False]:# , True]: # currently not working with nested recorder
            app_id = f"custom_app (use_parralel={use_parallel} use_app={use_app} use_recorder={use_recorder})"
            print(app_id)

            # Create custom app:
            ca = CustomApp(
                use_parallel=use_parallel,
                comp_kwargs={
                    CustomAgent: {"use_app": use_app, "use_recorder": use_recorder}
                },
            )

            # Create trulens wrapper:
            ta = TruCustomApp(
                ca,
                app_id=app_id,
            )

            # Sync usage
            print("  sync")
            print("    non-streaming")
            print("      untraced\t", ca.respond_to_query(query="What is the capital of Indonesia?"))
            with ta as recorder:
                print("      traced\t", ca.respond_to_query(query="What is the capital of Indonesia?"))

            # Sync streaming usage 
            print("    streaming")
            print("      untraced\t ", end="")
            for chunk in ca.stream_respond_to_query(query="What is the capital of Indonesia?"):
                print(chunk, end="")
            print()
            with ta as recorder:
                print("      traced\t ", end="")
                for chunk in ca.stream_respond_to_query(query="What is the capital of Indonesia?"):
                    print(chunk, end="")
                print()

            # TOFIX: Async usage
            #res1 = await ca.arespond_to_query(query="What is the capital of Indonesia?")
            #with ta as recorder:
            #    resp2 = await ca.arespond_to_query(query="What is the capital of Indonesia?")

            # TOFIX: Async streaming usage...
